# Mosaic Sentinel Spectral Features

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-04-07

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Mosaic Sentinel Spectral Features
# Author: Timm Nawrocki
# Created on: 2019-04-07
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Mosaic Sentinel Spectral Features" fills missing data and merges band and metric tiles into single band surfaces.

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
from arcpy.sa import *
import datetime
import os
import time

# Set root directory
drive = 'F:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input data
mask_raster = os.path.join(root_directory, 'Data_Input/area_of_interest/NuyakukAleknagik_Mask.tif')
tiles_project = os.path.join(root_directory, 'Data_Input/source_data/imagery/sentinel2/tiles_project')

# Define output directories
output_path = os.path.join(root_directory, 'Data_Input/predictor_img/mosaic_partial')

# Define the output mosaic names
month_names = ['05May',
               '06June',
               '07July',
               '08August',
               '09September',
               '10October'
              ]
spectral_metrics = ['1_ultraBlue',
                    '2_blue',
                    '3_green',
                    '4_red',
                    '5_redEdge1',
                    '6_redEdge2',
                    '7_redEdge3',
                    '8_nearInfrared',
                    '8a_redEdge4',
                    '11_shortInfrared1',
                    '12_shortInfrared2',
                    'nbr',
                    'ndmi',
                    'ndsi',
                    'ndvi',
                    'ndwi'
                   ] 
output_names = []
for month in month_names:
    for metric in spectral_metrics:
        set_name = month + '_' + metric
        output_names.append(set_name)
        
# Define projected tiles and store in dataframe for searchable access
projected_tiles = []
for file in os.listdir(tiles_project):
    if file.endswith('.tif'):
        projected_tiles.append(file)

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Mosaic and interpolate imagery

In [4]:
# Define a function to project and composite imagery tiles
def mosaic_interpolate(**kwargs):
    """
    Description: mosaics tiles to new raster, fills null values in image tiles based on nearest neighbor interpolation, and extracts to mask raster
    Inputs: 'input_array' -- an array containing the input rasters with the first raster as the mask raster and the additional rasters as the input tiles
            'output_array' -- an array of one output name
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_rasters = kwargs['input_array']
    mask_raster = input_rasters.pop(0)
    output_raster = kwargs['output_array'][0]
    output_path = os.path.split(output_raster)[0]
    # If output does not exist then process rasters
    if arcpy.Exists(output_raster) == False:
        print('----------')
        print('Processing raster tile {0}...'.format(os.path.split(output_raster)[1]))
        # Set the snap raster and cell size environments to match the area of interest
        arcpy.env.snapRaster = mask_raster
        arcpy.env.cellSize = mask_raster
        # If more than one input raster then mosaic rasters
        if len(input_rasters) >= 2:
            print('Merging tiles to new raster...')
            mosaic_raster = os.path.join(output_path, 'mosaic.tif')
            projection = arcpy.SpatialReference(3338) # NAD 1983 Alaska Albers
            arcpy.MosaicToNewRaster_management(input_rasters, os.path.split(mosaic_raster)[0], os.path.split(mosaic_raster)[1], projection, '32_BIT_FLOAT', '10', '1', 'LAST', 'FIRST')
        # If just one input raster then set the input equal to the mosaic raster
        elif len(input_rasters) == 1:
            mosaic_raster = input_rasters[0]
        # Create a null raster from the mosaic to identify no data cells
        print('Identifying no data cells...')
        null_raster = SetNull(IsNull(mosaic_raster), 1, "VALUE = 1")
        # Extrapolate missing data using nearest neighbors
        print('Filling no data cells based on extension of nearest neighbors...')
        filled_raster = Nibble(mosaic_raster, null_raster, 'DATA_ONLY', 'PROCESS_NODATA', '')
        # Extract filled raster to mask raster
        print('Extracting filled raster to mask...')
        extract_raster = ExtractByMask(filled_raster, mask_raster)
        # Copy extract raster to output
        print('Saving output...')
        arcpy.CopyRaster_management(extract_raster,
                                    output_raster,
                                    '',
                                    '',
                                    '-32768',
                                    'NONE',
                                    'NONE',
                                    '32_BIT_FLOAT',
                                    'NONE',
                                    'NONE',
                                    'TIFF',
                                    'NONE'
                                   )
        # Delete intermediate mosaic raster if it exists
        mosaic_raster = os.path.join(output_path, 'mosaic.tif')
        if arcpy.Exists(mosaic_raster):
            arcpy.Delete_management(mosaic_raster)
        # End timing function execution and calculate elapsed time
        end = time.time()
        elapsed = int(end - start)
        success_time = datetime.datetime.now()
        # Report process success
        print('Successfully merged and interpolated image...')
        print('----------')
        out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                    datetime.timedelta(seconds=elapsed))
        return out_process

In [5]:
# Run the mosaic and interpolation process for each tile set in list
for tile_set in output_names:
    # Define output raster
    output_raster = os.path.join(output_path, tile_set + '.tif')
    # Define input rasters
    input_rasters = []
    for tile in projected_tiles:
        if tile_set in tile:
            input_rasters.append(os.path.join(tiles_project, tile))
    # Define input and output arrays
    mosaic_interpolate_inputs = [mask_raster] + input_rasters
    mosaic_interpolate_outputs = [output_raster]
    # Create key word arguments
    mosaic_interpolate_kwargs = {'input_array' : mosaic_interpolate_inputs,
                                 'output_array' : mosaic_interpolate_outputs
                                }
    # Mosaic and interpolate image
    arcpy_geoprocessing(mosaic_interpolate, check_output = False, **mosaic_interpolate_kwargs)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
----------
Processing raster tile 10October_ndsi.tif...
Merging tiles to new raster...
Identifying no data cells...
Filling no data cells based on extension of nearest neighbors...
Extracting filled raster to mask...
Saving output...
Successfully merged and interpolated image...
----------
Succeeded at 2019-04-09 11:34 (Elapsed time: 0:04:29)
----------
Processing raster tile 10October_ndvi.tif...
Merging tiles to new raster...
Identifying no data cells...
Filling no data cells based on extension of nearest neighbors...
Extractin